<a href="https://colab.research.google.com/github/vignesh-pala/NLP/blob/master/Ch5_6/CtsNLP_Ch5_countmatrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Challenge 5** :  Assume you are looking to build a model to predict if the text you are seeing is an issue or informational message. For e.g. job completed is an informational message whereas job abended is an issue.  The objective is to predict the issue. 

 

The task is multi fold



1.   Generate the label for each data point to see if its informational or an issue
2.  Build a count based matrix for each issue which reflects the number of occurrence of each word in the text along with the label
3. You can choose to restrict to a certain number of words based on your analysis
4.You can choose to remove the STOPWORDS from each text 
5.The count based matrix that you build for the entire data set should be in a format such that it can be given as an input to any of the algorithm for training and testing.



In [93]:
import numpy as np
import re
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import pandas as pd
 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import metrics

from scipy.spatial import distance

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [94]:
df = pd.read_csv('/content/ticket_Data_labelled_input.csv',encoding='ISO-8859-1')
df.head()

,Sentence,Label
0,the job failed due to row count mismatch betwe...,0
1,Job got failed due to name pipe error.,0
2,Job got failed due to duplicate issue.,0
3,Job got failed due to file unavailability.,0
4,Job got failed due to tdata connectivity issue.,0


In [95]:
text = df['Sentence']
#stop_words = set(stopwords.words('english'))

In [96]:
count_vect = CountVectorizer(stop_words ='english' )
vector = count_vect.fit_transform(text)

In [97]:
print(count_vect.vocabulary_)


{'job': 320, 'failed': 245, 'row': 526, 'count': 147, 'mismatch': 392, 'source': 571, 'file': 252, 'ods': 416, 'table': 615, 'got': 268, 'pipe': 437, 'error': 224, 'duplicate': 208, 'issue': 312, 'unavailability': 664, 'tdata': 625, 'connectivity': 131, 'hadoop': 269, 'database': 162, 'named': 403, 'data': 160, 'log': 364, 'time': 638, 'executed': 234, 'completed': 117, 'successfully': 601, 'cancelled': 82, 'day': 168, 'records': 479, 'deleteing': 182, 'invalid': 308, 'collect': 106, 'stats': 582, 'killed': 331, 'hanged': 272, 'long': 366, 'manually': 383, 'format': 260, 'login': 365, 'null': 411, 'value': 691, 'running': 532, 'aborted': 12, 'tbload': 621, 'successfulll': 599, 'releasing': 493, 'process': 450, 'state': 581, 'failure': 246, 'query': 464, 'outage': 424, 'unailability': 663, 'abended': 10, 'connection': 129, 'low': 372, 'disk': 200, 'space': 573, 'started': 580, 'jobs': 322, 'values': 692, 'execution': 236, 'incorrect': 295, 'stream': 588, 'closure': 102, 'timeeout': 640,

In [98]:
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray()[0])

(9495, 713)
<class 'scipy.sparse.csr.csr_matrix'>
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

**Challenge 6:**
This is an extension to the Challenge 5 .  Assume you are looking to build a model to predict if the text you are seeing is an issue or informational message. For e.g. job completed is an informational message whereas job abended is an issue.  The objective is to predict the issue.

 

Build a TF-IDF based matrix along with the label.

Explore giving that input into a logistic regression to predict if the given text is an issue

Split Train, Test

In [99]:
X = df['Sentence']
y = df['Label']

X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)

**Build pipeline to vectorize data, train and fit model**

**Lets try CountVectorizer first**

In [100]:
#Naive Bayes:
txt_clf_nb = Pipeline([('tfidf', CountVectorizer()), 
                       ('clf', MultinomialNB())])

#LinearSVC:
txt_clf_lsvc = Pipeline([('tfidf', CountVectorizer()),
                         ('clf', LinearSVC())])

Naive Bayes Test

In [101]:
txt_clf_nb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [102]:
pred = txt_clf_nb.predict(X_test)

In [103]:
print(metrics.confusion_matrix(y_test, pred))

# Print the overall accuracy
print("Accuracy of Naive Bayes with CountVectorizer : ",metrics.accuracy_score(y_test,pred))

[[ 847   23]
 [ 181 2083]]
Accuracy of Naive Bayes with CountVectorizer :  0.9349074664964901


In [104]:
print(metrics.classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.82      0.97      0.89       870
           1       0.99      0.92      0.95      2264

    accuracy                           0.93      3134
   macro avg       0.91      0.95      0.92      3134
weighted avg       0.94      0.93      0.94      3134



LinearSVC pipeline

In [105]:
txt_clf_lsvc.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1000,
                           multi_class='ovr', penalty='l2', random_state=None,
                           tol=0.00

In [106]:
pred = txt_clf_lsvc.predict(X_test)

In [107]:
print(metrics.confusion_matrix(y_test, pred))

# Print the overall accuracy
print("Accuracy of Linear svc with CountVectorizer : ",metrics.accuracy_score(y_test,pred))

[[ 859   11]
 [   1 2263]]
Accuracy of Linear svc with CountVectorizer :  0.99617102744097


In [108]:
print(metrics.classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       870
           1       1.00      1.00      1.00      2264

    accuracy                           1.00      3134
   macro avg       1.00      0.99      1.00      3134
weighted avg       1.00      1.00      1.00      3134



**Summary of CountVectorizer**


*   Accuracy of Naive Bayes with CountVectorizer :  **0.9349074664964901**
*   Accuracy of Linear svc with CountVectorizer   :  **0.99617102744097**



**Lets try TfidfVectorizer first**

In [109]:
#Naive Bayes:
txt_clf_nb = Pipeline([('tfidf', TfidfVectorizer()), 
                       ('clf', MultinomialNB())])

#LinearSVC:
txt_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer()),
                         ('clf', LinearSVC())])

In [110]:
txt_clf_nb.fit(X_train, y_train)
pred = txt_clf_nb.predict(X_test)
print(metrics.confusion_matrix(y_test, pred))

# Print the overall accuracy
print("Accuracy of Naive Bayes with TfidfVectorizer : ",metrics.accuracy_score(y_test,pred))

[[ 835   35]
 [ 111 2153]]
Accuracy of Naive Bayes with TfidfVectorizer :  0.9534141671984684


In [111]:
txt_clf_lsvc.fit(X_train, y_train)
pred = txt_clf_lsvc.predict(X_test)
print(metrics.confusion_matrix(y_test, pred))

# Print the overall accuracy
print("Accuracy of Naive Bayes with TfidfVectorizer : ",metrics.accuracy_score(y_test,pred))

[[ 861    9]
 [   2 2262]]
Accuracy of Naive Bayes with TfidfVectorizer :  0.9964901084875558


**Summary of CountVectorizer**


*   Accuracy of Naive Bayes with TfidfVectorizer :  **0.9534141671984684**
*   Accuracy of Naive Bayes with TfidfVectorizer :  **0.9964901084875558**